In [1]:
import sys
sys.path.append('..')

In [2]:
from copy import deepcopy
import random
from library.problems.tsp import TSPSolution
from library.problems.data.tsp_data import distance_matrix
from library.algorithms.genetic_algorithms.algorithm import genetic_algorithm
from library.algorithms.genetic_algorithms.mutation import swap_mutation
from library.algorithms.genetic_algorithms.selection import fitness_proportionate_selection

ModuleNotFoundError: No module named 'library.problems'

## Specialized Genetic Operators

In the previous notebook, we explored some traditional genetic operators: **standard crossover, binary mutation, and swap mutation**.

However, these traditional operators generate invalid solutions when applied to combinatorial problems such as the Traveling Salesman Problem (TSP), job scheduling, or vehicle routing, where solutions are represented as **permutations**.

In this notebook, we’ll explore **specialized genetic operators**, specifically designed to handle permutations without producing invalid solutions. We’ll explore one crossover and one mutation methods that respects permutation constraints.

### Why Standard Crossover/Mutation Fail for Permutation Problems

In permutation-based problems (e.g., [1, 2, 3, 4, 5]), each gene must appear exactly once.
Standard genetic operators like one-point crossover or value-flip mutation can break this rule, resulting in invalid offspring with duplicates or missing values.


#### Standard Mutation - Value Flip

This mutation is inspired by the standard binary mutation: each gene is randomly replaced with another value with some probability.

Individual [1, 2, 3, 4, 5]

Mutated individual (hypothetical): [1, 2, 4, 1, 5] ❌ (Duplicate '1')

#### Standard Crossover

Parent 1: [1, 2 | 3, 4, 5]

Parent 2: [3, 4 | 5, 1, 2]


Offspring 1 (invalid): [1, 2, 5, 1, 2] ❌ (Duplicates '1' and '2')

Offspring 2 (invalid): [3, 4, 3, 4, 5] ❌ (Duplicates '3' and '4')

### Cycle Crossover

Cycle Crossover keeps items in their original positions across parents by identifying cycles of indices where elements should remain fixed.

**Pseudo-code:**

1. Choose random index in Parent 1 and copy the element to first child.
3. Copy element in same index in Parent 2 to second child.
4. Find this element in Parent 1 and copy it to first child, and repeat the process.
5. Once the cycle completes (we end up back on the initial index), the remaining positions are filled from the other parent.

![Cycle Crossover](images/cycle-xo.png)


In [ ]:
def cycle_crossover(parent1_repr: str | list, parent2_repr: str | list):
    random_initial_idx = random.randint(0, len(parent1_repr) - 1)

    cycle_idxs = [random_initial_idx]
    current_cycle_idx = random_initial_idx

    while True:
        value_parent2 = parent2_repr[current_cycle_idx]
        next_cycle_idx = parent1_repr.index(value_parent2)

        if next_cycle_idx == random_initial_idx:
            break
    
        cycle_idxs.append(next_cycle_idx)
        current_cycle_idx = next_cycle_idx

    offspring1_repr = []
    offspring2_repr = []

    for idx in range(len(parent1_repr)):
        if idx in cycle_idxs:
            offspring1_repr.append(parent1_repr[idx])
            offspring2_repr.append(parent2_repr[idx])
        else:
            offspring1_repr.append(None)
            offspring2_repr.append(None)

    if isinstance(parent1_repr, str) and isinstance(parent2_repr, str):
        offspring1_repr = ''.join(offspring1_repr)
        offspring2_repr = ''.join(offspring2_repr)

    return offspring1_repr, offspring2_repr

    


Let's test it.

In [ ]:
parent1 = [1, 2, 3, 4, 5, 6, 7]
parent2 = [3, 4, 2, 1, 7, 5, 6]

print("Parent 1:", parent1)
print("Parent 2:", parent2)

offspring1_repr, offspring2_repr = cycle_crossover(parent1_repr=parent1, parent2_repr=parent2)

print("Offspring 1:", offspring1_repr)
print("Offspring 2:", offspring2_repr)

### Inversion Mutation

Inversion mutation works by selecting two random indices and reversing the subsequence between them, with a certain probability.

![Inversion Mutation](images/inversion-mutation.png)

In [ ]:
def inversion_mutation(representation: str | list, mut_prob):
    first_idx.random.randint(0, len(representation) - 1)
    
    second_idx = first_index
    while abs(second_idx - first_idx) < 3:
        second_idx = random.randint(0, len(representation) - 1)
    
    #Make sure that first_idx is less than second_idx
    if first_idx > second_idx:
        first_idx, second_idx = second_idx, first_idx

    reversed_subsequence = list(reversed(representation[first_idx:second_idx]))

    if isinstance(representation, str):
        reversed_subsequence = ''.join(reversed_subsequence)

    new_repr = representation[:first_idx] + reversed_subsequence + representation[second_idx]

    return new_repr

else:
    return deepcopy(representation)  

    

    

    
    pass

SyntaxError: invalid syntax (2460012911.py, line 20)

Let's test it.

In [ ]:
representation = [1, 2, 3, 4, 5, 6, 7]

inversion_mutation(representation, mut_prob=1)

## Solving TSP with Genetic Algorithms

Just like we did in the previous notebook for the Knapsack Problem, we’ll now solve TSP using genetic algorithms.

To structure our solution, we’ll define a `TSPGASolution` class where we’ll define in the crossover and mutation methods needed to run the `genetic_algorithm` function.

To keep the class flexible and reusable, we’ll pass the `mutation_function` and `crossover_function` as callable arguments when creating an instance of `TSPGASolution`.

### A small but important side note

TSP solutions are represented as permutations of city indices, where the path must start and end at the same city (i.e., the starting index is fixed at both ends).

When applying permutation-based operators like cycle crossover or inversion mutation, we need to preserve this constraint. That means we should only apply genetic operators to the middle portion of the route: excluding the first and last cities, which must remain the same.

So in practice, our crossover and mutation functions will only operate on the inner part of the individual, keeping the boundaries intact.

In [ ]:
class TSPGASolution(TSPSolution):
    def __init__(
        self,
        mutation_function,
        crossover_function,
        distance_matrix,
        starting_idx,
        repr = None
            
    ):
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function

        super().__init__(
            distance_matrix=distance_matrix,
            starting_idx=starting_idx,
            repr=repr
        )

    def mutation(self, mut_prob):
        middle_segment = self.repr[1:-1]
        mutated_segment = self.mutation_function(middle_segment, mut_prob)

        new_repr = [self.starting_idx] + mutated_segment + [self.starting_idx]

        return TSPGASolution(
            repr=new_repr
            mutation_function = self.mutation_function
            crossover_function= = self.crossover_function,
            distance_matrix = self.distance_matrix,
            starting_idx=self.starting_idx
        )

    def crossover(self, other_solution):
        parent1_middle_segment = self.repr[1:-1]
        parent2_middle_segment = other_solution.repr[1:-1]

        offspring1_middle, offspring2_middle = self.crossover_function(parent1_middle_segment, parent2_middle_segment)

        offspring1_repr = [self.starting_idx] + offspring1_middle + [self.starting_idx]
        offspring2_repr = [self.starting_idx] + offspring2_middle + [self.starting_idx]

        return TSPGASolution(
            repr = offspring1_repr,
            mutation_function = self.mutation_function
            crossover_function= = self.crossover_function,
            distance_matrix = self.distance_matrix,
            starting_idx=self.starting_idx
        ),
        TSPGASolution(
            repr = offspring2_repr,
            mutation_function = self.mutation_function
            crossover_function= = self.crossover_function,
            distance_matrix = self.distance_matrix,
            starting_idx=self.starting_idx
        )
        pass

Let's apply the genetic algorithm to TSP using cycle crossover and inversion mutation. Here the probability of mutation should be relatively small because inversion mutation may be very destructive.

In [ ]:
# TODO: Apply GA to TSP with cycle crossover and inversion mutation

And now let's use cycle crossover again, but this time we use swap mutation.
Now we can set a higher probability of miutation because swap mutation is less destructive than inversion mutation.

In [ ]:
# TODO: Apply GA to TSP with cycle crossover and swap mutation